# The Art of Prolog
1986年MIT Pressから出版されたPrologの教科書 The Art of Prolog は、私の前職である株式会社 構造計画研究所から1988年1月1日に「Prologの技芸」というタイトルで日本語訳が出版されました。

残念ながらPrologの技芸は絶版となり、Amazonの古本でも12,800円と高額で販売されています。

幸運なことに、英語の原本の方は、MIT Book pressのサーバからダウンロードすることができます。
- https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/1407/1407.pdf?dl=1

2024年になって、推論エンジンの原理と応用を再度確認するために、「Prologの技芸」の例題をpythonにポーティングした推論エンジンと簡易パーサを使ってトレースしました。

さらに、swi prologを使って、23章の「コンパイラ」をトレースしました。以下のノートはその時の記録に加筆し、整理したものです。

## ノートブック実行の注意
最初に、コマンドパレットを表示し、"Select Notebook Kernel"を実行し、"Prolog"を選択します。
これで、code cellの内容がswi prologによって処理されます。

## 差分リストの表現
23章のコンパイラーでは、16章に出てくる差分リスト表現（X\Y）が使われていいますが、これを[X, Y]としても実行可能かどうかを試します。

差分リストのメリットは、リストのコピーが不要で、tail(尾部)は、ポインターとして働くことです。

## １６章　限定節文法による構文解析

### 文脈自由文法
16.1の文脈自由文法を定義する。

終端記号はカギ括弧で括る。
文法がさりげなく、右再帰構文になっていることに注意！

In [3]:
% Figure 16.1 簡単な文脈自由文法
sentence --> noun_phrase, verb_phrase.
noun_phrase --> determiner, noun_phrase2.
noun_phrase --> noun_phrase2.
noun_phrase2 --> adjective, noun_phrase2.
noun_phrase2 --> noun.
verb_phrase --> verb.
verb_phrase --> verb, noun_phrase.
% 語彙
determiner --> [the].   adjective --> [decorated].
determiner --> [a].
noun --> [pieplate].    verb --> [surprise].
noun --> [surprise].

Previously defined clauses of user:sentence/2 were retracted:
:- dynamic sentence/2.

sentence(A, C) :-
    noun_phrase(A, B),
    verb_phrase(B, C).

% Asserting clauses for user:sentence/2


Previously defined clauses of user:noun_phrase/2 were retracted:
:- dynamic noun_phrase/2.

noun_phrase(A, C) :-
    determiner(A, B),
    noun_phrase2(B, C).
noun_phrase(A, B) :-
    noun_phrase2(A, B).

% Asserting clauses for user:noun_phrase/2


Previously defined clauses of user:noun_phrase2/2 were retracted:
:- dynamic noun_phrase2/2.

noun_phrase2(A, C) :-
    adjective(A, B),
    noun_phrase2(B, C).
noun_phrase2(A, B) :-
    noun(A, B).

...

% Asserting clauses for user:adjective/2


Previously defined clauses of user:noun/2 were retracted:
:- dynamic noun/2.

noun([pieplate|A], A).
noun([surprise|A], A).

% Asserting clauses for user:noun/2


Previously defined clauses of user:verb/2 were retracted:
:- dynamic verb/2.

verb([surprise|A], A).

% Asserting clauses for user:verb/2


分脈自由文法は、S\S0の差分リストを引数とする。prolog文に変換される。
```prolog
sentence --> noun_phrase, verb_phrase.
```
これをそのままprolog規則に変換すると
```prolog
sentence(S) :- append(NP, VP, S), noun_pharase(NP), verb_phrase(VP).
```
となるが、appendの処理効率がわるいため、差分リストで表すと以下の様になる。
```prolog
sentence(S\S0) :- noun_phrase(S\S1), verb_phrase(S1\S0).
```
さらにAs\Bsは、headとtailのリストに分解されるので、
```prolog
sentence(S, S0) :- noun_phrase(S, S1), verb_phrase(S1, S0).
```
となる。

listing関数を使って、sentenceの変換結果を見ると、上記の変換が行われていることが確認できる。

In [4]:
?- listing(sentence).

:- dynamic sentence/2.

sentence(A, C) :-
    noun_phrase(A, B),
    verb_phrase(B, C).


true

### 文法規則からProlog節への変換
swi-prologは、すでに文法規則からProlog節への変換が組み込まれていますが、Program 16.2の例題の動作をswi-prologで動かしてみます。

sequenceはswi-prologのライブラリをそのまま使用します。

In [6]:
% translate(GrammerRule, PrologClause) :-
%   PrologClauseは、文脈自由文法の規則GrammerRuleと等価なPrologプログラムである。
translate((Lhs --> Rhs), (Head :- Body)) :- 
    translate(Lhs, Head, [Xs, Ys]), translate(Rhs, Body, [Xs, Ys]).
translate((A, B), (A1, B1), [Xs, Ys]) :- 
    translate(A, A1, [Xs, Xs1]), translate(B, B1, [Xs1, Ys]).
translate(A, A1, S) :-
    non_terminal(A), functor(A1, A, 1), arg(1, A1, S).
translate(Xs, true, S) :-
    terminals(Xs), sequence(Xs, S).
non_terminal(A) :- atom(A).
terminals([X|Xs]).
%sequence([X|Xs], [[X|S], S0]) :- sequence(Xs, [S, S0]).
%sequence([], [Xs, Xs]).

Previously defined clauses of user:translate/2 were retracted:
:- dynamic translate/2.

translate((A-->C),  (B:-D)) :-
    translate(A, B, [E, F]),
    translate(C, D, [E, F]).

% Asserting clauses for user:translate/2


Previously defined clauses of user:translate/3 were retracted:
:- dynamic translate/3.

translate((A, D),  (B, E), [C, G]) :-
    translate(A, B, [C, F]),
    translate(D, E, [F, G]).
translate(A, B, C) :-
    non_terminal(A),
    functor(B, A, 1),
    arg(1, B, C).
translate(A, true, B) :-
    terminals(A),
    sequence(A, B).

% Asserting clauses for user:translate/3


Previously defined clauses of user:non_terminal/1 were retracted:
:- meta_predicate non_terminal(:).

system:non_terminal(Spec) :-
    '$set_pattr'(Spec, pred, non_terminal(true)).

%   NOTE: system definition has been overruled for non_terminal/1
:- dynamic non_terminal/1.

non_terminal(A) :-
    atom(A).

% Asserting clauses for user:non_terminal/1


Previously defined clauses of user:terminals/1 were retracted:
:- dynamic terminals/1.

terminals([_|_]).

% Asserting clauses for user:terminals/1


動作確認として、以下の自由文脈文法をProlog節に変換してみます。
```
s --> n, v
```

In [7]:
?- translate((s --> n, v), (H :- B)), print(H), print(:-), print(B).

s([_9348,_9354]):-n([_9348,_9388]),v([_9388,_9354])

H = s([_9348,_9354]),
B = n([_9348,_9388]),v([_9388,_9354])

出力されたProlog節を実行し、listing関数で表示してみると、期待通り以下の出力が返されます。
```
s([A, C]) :- n([A, B]), v([B, C]).

In [8]:
s([_9348,_9354]):-n([_9348,_9388]),v([_9388,_9354])

% Asserting clauses for user:s/1


In [9]:
?- listing(s).

:- dynamic s/1.

s([A, C]) :-
    n([A, B]),
    v([B, C]).


true

### 差分リストの表現
15章のProgram 15.2では差分リストを使って、以下のようにappend_dlが定義されています。

```prolog
append_dl(As, Bs, Cs) :- append_dl(Xs\Ys, Ys\Zs, Xs\Zs).
```

このX\Yの部分を[X, Y]に置き換えて、動きを見てみます。

In [1]:
append_dl([Xs, Ys], [Ys, Zs], [Xs, Zs]).
?- append_dl([[a, b, c|Xs], Xs], [[1, 2], []], Ys).

% Asserting clauses for user:append_dl/3


Xs = [1,2],
Ys = [[a,b,c,1,2],[]]

テキストの本文に?- append_dl([a, b, c|Xs]\Xs, [1, 2]\[], Ys)の結果が(Xs=[1,2], Ys=[a, b, c, 1, 2]\[])と記述があります。

これは上記の結果と合致しています。